In [3]:
# !pip install mysql-connector-python

In [9]:
## Reading From init file
from mysql_connect import MySqlConnection

In [2]:
import pandas as pd
import mysql.connector
import getpass,os

## Common Functions

In [3]:
def dir_update(txt):
    txt = txt.replace("\\","/")
    return txt

In [4]:
dir_update(os.getcwd())

'd:/ML_NOTE/Data Analysis/Data_Acquisition'

## MySqlConnection Class

In [69]:
class MySqlConnection:
    def __init__ (self,host,user,database):
        self.host = host
        self.user = user
        self.password =  self.password = getpass.getpass(prompt="Enter your password: ")
        self.database = database
        self.conn = None
        

    
    def connect(self):
        """Establish Mysql Connection.."""
        try:
            self.conn = mysql.connector.connect(host = self.host,
                                                user = self.user,
                                                password = self.password,
                                                database = self.database,)
            if self.conn.is_connected():
                print(f"MySql Connection Details:\nData Base --> {self.database} ")

                cursor = self.conn.cursor()
                cursor.execute("select user();")
                print(f"User --> {cursor.fetchone()[0]}")

                cursor.execute("SELECT DATE_FORMAT(NOW(), '%Y-%m-%d %r') AS CurrentDateTime;")
                print(f"Date Time --> {cursor.fetchone()[0]}")
    
        except Exception as e:
            print(f"MySql Connect failed .....{e}")
            self.conn = None

            
    def table_query(self,query):
        """Get records from Mysql into the Dataframe"""
        try:
            if self.conn is None or not self.conn.is_connected():
                print("Reconnecting to MySQL...")
                self.connect()
                
            cursor = self.conn.cursor()
            cursor.execute(query)
            records = cursor.fetchall()
            cursor.close()
            
            column_list =[desc[0] for desc in  cursor.description]
            df = pd.DataFrame(records, columns=column_list)
            return df
        except  mysql.connector.Error as e:
            print(f"Error reading data from MySQL: {e}")
            return None
            
    
    def create_table(self, create_table_query, table_name):
        """Create a table in the database"""
        try:
            cursor = self.conn.cursor()
            cursor.execute(create_table_query)
            print(f"Table [{table_name}] created successfully.")
            cursor.close()
        except Exception as e:
            print(f"Error creating table: {e}")


    def insert_record_from_df (self, table_name, df):
        try:  
            if self.conn is None or not self.conn.is_connected():
                print("Reconnecting to MySQL...")
                self.connect()
            cursor = self.conn.cursor()
            for i, row in df.iterrows():
                # Create an INSERT query for each row
                row_values = row.astype(str).values.tolist()
                
                placeholders = ", ".join(["%s"] * len(row))
                columns = ", ".join(row.index)
                insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                cursor.execute(insert_query, tuple(row_values))
            self.conn.commit()
            print(f"Inserted {len(df)} rows into {table_name} successfully.")
            cursor.close()
        except Exception as e:
            print(f"Error inserting data: {e}")
            self.conn.rollback()
            cursor.close()

    
    def connection_close(self):
        if self.conn:
            self.conn.close()
            print('MySql Connection Closed.........\n')
 
        


In [8]:
budget_schema_config = {
    "host":"127.0.0.1",
    "user":"analysis",
    "database":'myhr'
}

mysql_conn = MySqlConnection(**budget_schema_config)
mysql_conn.connect()

MySql Connection Details:
Data Base --> myhr 
User --> analysis@localhost
Date Time --> 2024-12-23 01:16:02 PM


In [71]:
employee_df  = mysql_conn.table_query("Select * from employees")
employee_df

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id
0,100,Steven,King,steven.king@sqltutorial.org,515.123.4567,1987-06-17,4,24000.00,NaN,9
1,101,Neena,Kochhar,neena.kochhar@sqltutorial.org,515.123.4568,1989-09-21,5,17000.00,100.0,9
2,102,Lex,De Haan,lex.de haan@sqltutorial.org,515.123.4569,1993-01-13,5,17000.00,100.0,9
3,103,Alexander,Hunold,alexander.hunold@sqltutorial.org,590.423.4567,1990-01-03,9,9000.00,102.0,6
4,104,Bruce,Ernst,bruce.ernst@sqltutorial.org,590.423.4568,1991-05-21,9,6000.00,103.0,6
5,105,David,Austin,david.austin@sqltutorial.org,590.423.4569,1997-06-25,9,4800.00,103.0,6
6,106,Valli,Pataballa,valli.pataballa@sqltutorial.org,590.423.4560,1998-02-05,9,4800.00,103.0,6
7,107,Diana,Lorentz,diana.lorentz@sqltutorial.org,590.423.5567,1999-02-07,9,4200.00,103.0,6
8,108,Nancy,Greenberg,nancy.greenberg@sqltutorial.org,515.124.4569,1994-08-17,7,12000.00,101.0,10
9,109,Daniel,Faviet,daniel.faviet@sqltutorial.org,515.124.4169,1994-08-16,6,9000.00,108.0,10


In [31]:

mysql_conn.connection_close()

MySql Connection Closed.........

